# Event Time
This is an important topic to cover discretely because spark's Dstream API does not support processing information with respect to event time. At a higher level, in stream processing systems there are effectively two relevant times for each event: the time at which it actually occurred (event time), and the time that it was processes or reached the stream processing system (processing time)

**Event Time**
This is the time that is embedded in the data itself. It is most often, though not required to be, the time that an event actually occurs. Thgis is important to use because it provides a more robust way of comparing events against one another. The challenge here is that event data can be late or out of order. This means that the stream processing system must be able to handle out-of-order or late data

**Processing Time**
This is the time at which the stream-processing system actually received data. This is usually less important than event time because when it's processed is largely an implementation detail

# Stateful Processing
This is only necessary when you need to use or update intermediate information (state) over longer periods of time (in either a microbatch or a record-at-a-time approach). This can happen when you are using event time or when you are performing an aggregation on a key, whether that involves event time or not

# Arbitrary Stateful Processing
There are times when you need fine-grained control over what state should be stored, how it is updated, and when it should be removed, either explicitly or via a time-out. This is called arbitrary stateful processing and spark allows you to essentially store whatever information you like over the course of the processing of a stream. This provides immense flexibility and power and allows for some complex business logic to be handled quite easily. Here are some examples:
* You'd like to record information about user sessions on an ecommerce site. For instance, you might want to track what pages users visit over the course of this session in order to provide recommendations in real time during their next session. Naturally, these sessions have completely arbitrary start and stop times that are unique to that user
* Your company would like to report on errors in the web application but only if five events occur during a user's session. You could do this with count-based windows that only emit a result if five events of some type occur
* You'd like to deduplicate records over time. To do so, you're going to need to keep track of every record that you see before deduplicating it.

# Event-Time Basics

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/11/09 10:34:06 WARN Utils: Your hostname, kevin resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
22/11/09 10:34:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/09 10:34:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set('spark.sql.shuffle.partitions', '5')
static = spark.read.json('/home/kevin/Desktop/Big-Data-with-Pyspark/data/activity-data')
streaming = spark\
                .readStream\
                .schema(static.schema)\
                .option('maxFilesPerTrigger', 10)\
                .json('/home/kevin/Desktop/Big-Data-with-Pyspark/data/activity-data')

streaming.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In this dataset, there are two time-based columns. The creation_time column defines when an event was created, whereas the arrival_time defines when an event hit our servers somewhere upstream. 

# Windows on Event Time
The first step in event-time analysis is to convert the timestamp column into the proper spark SQL timestamp type. Our current column is unixtime nanoseconds (represented as long), therefore we're going to manipulate it

In [3]:
withEventTime = streaming.selectExpr(
    "*",
    "cast(cast(Creation_time as double)/1000000000 as timestamp) as event_time"
)


## Tumbling Windows
The simplest operation is simply to count the number of occurrences of an event in a given window. This depicts the process when performing a simple summation based on the input data and a key 
When performing an aggregation of keys over a window of time, we update the result table depending on the data received since the last trigger. In the case of our data, we will do so in intervals of 10 minute windows without any overlap between each (each, and only one event fall into one window). This will update in real time, as well, meaning that if new events were being added upstream to our system, structured streaming would update those counts accordingly. This is the complete output mode, spark will output the entiure result table regardless of whether we've seen the entire dataset

In [4]:
from pyspark.sql.functions import window, col

withEventTime.groupBy(window(col('event_time'), '10 minutes')).count()\
    .writeStream\
    .queryName('pyevents_per_window')\
    .format('memory')\
    .outputMode('complete')\
    .start()

22/11/09 10:49:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-095a9b1c-8954-4621-92e0-7fb7be1e612a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/09 10:49:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
spark.sql("SELECT * FROM pyevents_per_window").show(5)

+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-24 14:50...|150773|
|{2015-02-24 16:00...|133323|
|{2015-02-23 15:30...|100853|
|{2015-02-23 13:20...| 99178|
|{2015-02-24 15:30...|125679|
+--------------------+------+
only showing top 5 rows



In [6]:
spark.sql("SELECT * FROM pyevents_per_window").printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- count: long (nullable = false)



### Sliding Windows
We will run a sliding window through which we will look at an hour increment, but we'd like to get the state every 10 minutes. This means that we will update the values over time and we will include the last hours of data. In this example, we have 10-minute windows, starting every five minutes. Therefore, each event will fall into two different windows

In [9]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col('event_time'), '10 minutes', '5 minutes'))\
            .count()\
            .writeStream\
            .queryName('pyevents_per_slidingWindow')\
            .format('memory')\
            .outputMode('complete')\
            .start()

22/11/09 10:56:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bd111936-2d64-4bed-9243-20ca791be0bd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/09 10:56:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [10]:
spark.sql('SELECT * FROM pyevents_per_slidingWindow').show(5)

+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 17:15...|107668|
|{2015-02-24 14:50...|150773|
|{2015-02-24 16:00...|133323|
|{2015-02-22 03:35...|    35|
|{2015-02-23 15:30...|100853|
+--------------------+------+
only showing top 5 rows



## Handling Late Data with Watermarks
The preceding examples are grerat but we did not specify how late we expect to see the data. This means that spark is going to need to store that intermediate data forever because we never specified a watermark, or a time at which we don't expect to see any more data. We must specify this watermark in order to age-out data in the stream(and therefore, state) so that we don't overwhelm the system over a long period of time
A watermark is an amount of time following a given event or set of events after which we do not expect to see any more of data from that time. We know this can happen due to delays on the network, devices that lose connection etc. In the DStreams API, there was no robust way to handle late data in this way- if an event occurred at a certain time but did not make it to the processing system by the time the batch for a given window started, it would show up in other processing batches. 
Structured Streaming remedies this. In event and stateful processing, a given window's state or set of data is decoupled from a processing window. That means that as more events come in, Structured Streaming will continue to update a window with more information

If we know that we typically see data as produced downstream in minutes but we have seen delays in events up to 5 hours after they occur (perhaps the user lost cell phone connectivity), we'd specify the watermark in the following way:

In [11]:
withEventTime\
    .withWatermark('event_time', '30 minutes')\
    .groupBy(window(col('event_time'), '10 minutes', '5 minutes'))\
    .count()\
    .writeStream\
    .queryName('pyevents_per_window2')\
    .format('memory')\
    .outputMode('complete')\
    .start()

22/11/09 11:19:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0b62aa06-ea22-426c-ae14-69ea3b6e20d9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/09 11:19:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
spark.sql('SELECT * FROM pyevents_per_window2').show(5)

+--------------------+------+
|              window| count|
+--------------------+------+
|{2015-02-23 17:15...|107668|
|{2015-02-24 14:50...|150773|
|{2015-02-24 16:00...|133323|
|{2015-02-22 03:35...|    35|
|{2015-02-23 15:30...|100853|
+--------------------+------+
only showing top 5 rows



## Dropping Duplicates in a Stream
One of the more difficult operations in record-at-a-time systems is removing duplicates from the stream.Almost by definition, you must operate on a batch of records at a time in order to find duplicates - there's a high coordination overhead in the processing system. Deduplication is an important tool in many applications, especially when messages might be delivered multiple times by upstream systems. To deduplicate data, spark will maintain a number of user specified keys and ensure that duplicates are ignored.
Notice here that we need to specify the event time column as a duplicate column along with the column you should deduplicate. The core assumption is that duplicate events will have the same timestamp as well as identifier. In this model, rows with two different timestamps are two different records

In [13]:
from pyspark.sql.functions import expr

withEventTime\
    .withWatermark('event_time', '5 seconds')\
    .dropDuplicates(['User', 'event_time'])\
    .groupBy('User')\
    .count()\
    .writeStream\
    .queryName('pydeduplicated')\
    .format('memory')\
    .outputMode('complete')\
    .start()

22/11/09 11:36:59 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9886d512-a6b2-4622-9013-1c529411239b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/09 11:36:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


# Arbitrary Stateful Processing
We have seen hpw spark maintains information and updates windows based on our specifications. But things differe when you have more complex concepts of windows; this is, where arbitrary stateful processing comes in. This section includes several examples of different use cases along with examples that show you how you might go about setting up your business logic.

When performing stateful processing, you might want to do the following:
* Create window based on counts of a given key
* Emit an alert if there is a number of events within a certain time frame
* Maintain user sessions of an undetermined amount of time and save those sessions to perform some analysis on later

At the end of the day, there are two things you will want to do when performing this style of processing:
* Map over groups in your data, operate on each group of data, and generate at most a single row for each group. The relevant API for this use case is mapGroupsWithState
* Map over groups in your data, operate on each group of data, and generate one or more rows for eacyh group. The relevant API for this use case is flatMapGroupWithState



In [14]:
spark.stop()

22/11/09 11:46:46 WARN StateStore: Error running maintenance thread
java.lang.IllegalStateException: SparkEnv not active, cannot do maintenance on StateStores
	at org.apache.spark.sql.execution.streaming.state.StateStore$.doMaintenance(StateStore.scala:596)
	at org.apache.spark.sql.execution.streaming.state.StateStore$.$anonfun$startMaintenanceIfNeeded$1(StateStore.scala:582)
	at org.apache.spark.sql.execution.streaming.state.StateStore$MaintenanceTask$$anon$1.run(StateStore.scala:442)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.